In [1]:
from google.colab import drive
drive.mount("/content/drive")
import csv
import re
import tqdm.notebook as tq
import pickle
import pandas as pd

Mounted at /content/drive


In [14]:
# load data
data_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data/annot_data/annotated_data_bkup/20210412/aspect_tuples_20210412.1.csv'

with open(data_path, 'r', encoding='utf-8') as f:
  rows = csv.reader(f, delimiter = ',')
  headers = next(rows)
  all_data = [ row for row in rows ]
  evaltext = [ data[6] for data in all_data]
  evaltext_cntxt = [ data[6] for data in all_data if data[3] == 'True'] # filter evaltext whose is_context is True

construction_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/sentiment-construction-list/data/constructions_0415.csv'
with open(construction_path) as f:
    constructions = [ row.strip() for row in f ]

noise_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/data/annot_data/annotated_data_bkup/20210412/noise_pairs_20210412.pkl'
with open(noise_path, 'rb') as f:
  noise = pickle.load(f)
  noise_text = [ data[0] for data in noise ]

seeds_path = '/content/drive/Shareddrives/LOPE/產學合作/中華電信/2020-2021/指向情緒案/sentiment-construction-list/data/seeds.csv'
with open(seeds_path, 'r', encoding='utf-8') as f:
  rows = csv.reader(f, delimiter = ',')
  seeds = [ ' '.join(row) for row in rows ]
seeds_pat = "(%s)" % "|".join(seeds)

In [6]:
# find matched seeds in evaltext_cntxt
matched_seeds = []

for text in evaltext_cntxt:
  for seed in seeds:
    if seed in text:
      matched_seeds.append(seed)

matched_seeds_pat = re.compile("(%s)" % "|".join(matched_seeds))

In [7]:
# replace seeds in evaltext_cntxt with `__SLOT__` to create new constructions
all_new_cnstr = []
for text in evaltext_cntxt:
  new_cnstr = re.sub(matched_seeds_pat, '__SLOT__', text)
  all_new_cnstr.append(new_cnstr)

In [15]:
# find if new constructions are found in evaltext
matched_eval = {}
matched_eval_count = {}
for k in tq.tqdm(all_new_cnstr):
  ori_key = k
  k = re.escape(k)
  k = k.replace('__SLOT__', seeds_pat) # replace `__SLOT__` with the seed if there is any
  try:
    pat = re.compile(k)
  except:
    print(k)
    break
  
  # Get all cnstr from all comments
  candidates = set()
  for text in evaltext:
    for c in pat.finditer(text): candidates.add(c[0])
  
  # Count all cnstr
  matched_eval_count[ori_key] = len(candidates)

  if len(candidates) > 1:
    if ori_key not in matched_eval: matched_eval[ori_key] = []
    for c in candidates: matched_eval[ori_key].append(c)

In [22]:
# find if new constructions are found in noise_text
matched_noise = {}
matched_noise_count = {}
for k in tq.tqdm(all_new_cnstr):
  ori_key = k
  k = re.escape(k)
  k = k.replace('__SLOT__', seeds_pat) # replace `__SLOT__` with the seed if there is any
  try:
    pat = re.compile(k)
  except:
    print(k)
    break
  
  # Get all cnstr from all comments
  candidates = set()
  for text in noise_text:
    for c in pat.finditer(text): candidates.add(c[0])
  
  # Count all cnstr
  matched_noise_count[ori_key] = len(candidates)

  if len(candidates) > 1:
    if ori_key not in matched_noise: matched_noise[ori_key] = []
    for c in candidates: matched_noise[ori_key].append(c)

In [ ]:
# find if original constructions are found in noise_text
matched = {}
matched_con = {}
for k in tq.tqdm(constructions):
  pat = re.compile(k)
  if k not in matched: matched[k] = []
  
  # Get all cnstr from all comments
  candidates = set()
  for cmt in noise_text:
    for c in pat.finditer(cmt): candidates.add(c[0])
  
  # Count all cnstr
  matched_con[k] = len(candidates)

  if len(candidates) > 1:
    for c in candidates: matched[k].append(c)

In [29]:
df = pd.DataFrame([matched_eval_count, matched_noise_count]).T
df.columns = ['matches in all evaltext', 'matches in noise']
df = df.sort_values(by='matches in noise', ascending=False)
df['sum'] = df['matches in all evaltext'] + df['matches in noise']
df['ratio'] = df['matches in all evaltext'] / df['matches in noise']
df

,matches in all evaltext,matches in noise,sum,ratio
__SLOT__,209,474,683,0.440928
都__SLOT__,7,18,25,0.388889
__SLOT__11,3,10,13,0.300000
__SLOT__最快,1,3,4,0.333333
Hami pa,1,1,2,1.000000
...,...,...,...,...
__SLOT__電信的店員用服務贏了一個用戶,1,0,1,inf
可惜話術太差 輸__SLOT__大好幾年 高階機分潤越來越差 __SLOT__都不主打哀鳳了,1,0,1,inf
整體平衡我會首選__SLOT__再不然撿便宜也是__SLOT____SLOT__就不上不下沒有便宜也沒有__SLOT__，整個很糟糕,1,0,1,inf
以__SLOT__ 來說__SLOT__>__SLOT__>Others,1,0,1,inf


In [30]:
df.to_csv('results.csv')